In [1]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [2]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import utils

Using TensorFlow backend.


In [3]:
sequence = sorted([p for p in CELLS])

In [4]:
a = np.array([[1]*39])
b = [500]

In [5]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(128,activation='relu', input_shape=(39,)))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [6]:
model = build_model()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
try:
    model = load_model('my_model.h5')
except:
    print("none model found")
    model = build_model()

Instructions for updating:
Use tf.cast instead.


In [8]:
!ls '../rec'

'[1win]1556524801.0caobluejiba.txt'   '[2zzz]1556524823.0caogreenjiba.txt'
'[1win]1556524809.0caobluejiba.txt'   '[2zzz]1556524823.0caoredjiba.txt'
'[1win]1556524823.0caobluejiba.txt'   '[2zzz]1556524827.0caobluejiba.txt'
'[1win]1556524827.0caoredjiba.txt'    '[2zzz]1556524827.0caogreenjiba.txt'
'[2zzz]1556524801.0caogreenjiba.txt'  '[2zzz]1556524977.0caobluejiba.txt'
'[2zzz]1556524801.0caoredjiba.txt'    '[2zzz]1556524977.0caoredjiba.txt'
'[2zzz]1556524809.0caogreenjiba.txt'  '[2zzz]1556525620.0caobluejiba.txt'
'[2zzz]1556524809.0caoredjiba.txt'    '[2zzz]1556525620.0caoredjiba.txt'


In [9]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [10]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [11]:
def load_logs():
    datapath = join(join_list(os.getcwd().split('/')[:-1]),'rec/')
    files = bash_call(['ls', datapath])
    return files

In [17]:
load_logs()

[]

In [13]:
def preprocess_log(filename, condition=None):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    print(filename)
    
    states, color = utils.load_l('../rec/'+filename)
    
    train = []
    value = []
    
    for s in states:
        board = []
        # add board
        for cord in sequence:
            board.append(s['board'][cord])
        # color 
        board.append(color)
        board.append(s['d_heur'])
        train.append(board)
        value.append(s['rew'])
    
    value.reverse()
    decay = 0.98
    curr_decay = 0.98
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
    
    value.reverse()
    
    return train, value

In [14]:
preprocess_log('[2zzz]1556525620.0caoredjiba.txt')

[2zzz]1556525620.0caoredjiba.txt


([[2,
   2,
   2,
   0,
   0,
   0,
   0,
   0,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   3,
   1,
   0,
   0,
   0,
   0,
   3,
   1,
   0,
   0,
   0,
   3,
   1,
   0,
   0,
   3,
   2,
   -1],
  [2,
   2,
   0,
   2,
   0,
   0,
   0,
   0,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   3,
   1,
   0,
   0,
   0,
   3,
   0,
   1,
   0,
   0,
   0,
   3,
   1,
   0,
   0,
   3,
   2,
   0],
  [2,
   2,
   0,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   2,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   3,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   3,
   3,
   1,
   0,
   0,
   3,
   2,
   -1],
  [0,
   2,
   0,
   2,
   0,
   2,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   2,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   3,
   0,
   1,
   0,
   3,
   3,
   1,
   0,
   0,
   3,
   2,
   -1],
  [0,
   2,
   0,
   0,
   0,
   2,
   0,
   2,
   0,
   

In [16]:
logfs = load_logs()
for logf in logfs:
    train, value = preprocess_log(logf, '2zzz')
    if train:
        print(len(train[-1]), len(value))
        model.fit(train, value, epochs=1, batch_size=1)

[2zzz]1556524801.0caogreenjiba.txt


TypeError: list indices must be integers or slices, not str

In [ ]:
res[-1]

In [ ]:
target

In [ ]:
n = model.predict(np.array([res[-1]]))
n

In [ ]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')


In [ ]:
def save_model():
    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('trained_model', 'wb') as f:
        pickle.dump(model.get_weights(), f)

In [ ]:
save_model()

In [ ]:
model.predict(np.array([[1]*39,[0.5]*39]))